In [635]:
import pandas as pd

In [636]:
DATA_FILE = 'BalanceSPL_Fonction_2018_Dec2019.csv'

In [637]:
data = pd.read_csv(DATA_FILE, sep=';')

In [638]:
### Mise en forme

def string_to_float(ligne):
    return float(ligne.replace(',','.'))

COLS = ['IDENT', 'NDEPT', 'INSEE', 'siren', 'LBUDG', 'COMPTE', 'OOBDEB', 'OOBCRE',
        'FONCTION', 'SD', 'SC', 'OBNETDEB', 'OBNETCRE']
FLOAT_COLS = ['SD', 'SC', 'OBNETDEB', 'OBNETCRE', 'OOBDEB', 'OOBCRE']

data = data[COLS]
for col in FLOAT_COLS:
    data[col] = data[col].apply(string_to_float)
    
data['COMPTE'] = data['COMPTE'].apply(str)

In [639]:
AGGREGATS = {
    1: 'Achats et charges externes', 
    2: 'Impôts et taxes', 
    3: 'Charges de personnel',
    4: 'Autres charges de gestion courante',
    5: 'Charges financieres',
    6: 'Charges exceptionnelles',
    7: 'Depenses directes d investissement',
    8: 'Subventions d equipement versees',
    9: 'Prises de participation',
    10: 'Prets accordes',
    11: 'Remboursement d emprunts et de dettes assimilees',
}

In [640]:
DEFINITION_AGGREGATS = {
    1: {'a_prendre': ['60', '61', '62'], 'a_exclure': ['621']},
    2: {'a_prendre': ['63'], 'a_exclure': ['631', '633']}, 
    3: {'a_prendre': ['64', '621', '631', '633'], 'a_exclure': []},        
    4: {'a_prendre': ['65'], 'a_exclure': []},
    5: {'a_prendre': ['66'], 'a_exclure': []},
    6: {'a_prendre': ['67'], 'a_exclure': ['675', '676']},
    7: {'a_prendre': ['20', '21', '23'], 'a_exclure': ['204']},
    8: {'a_prendre': ['204'], 'a_exclure': []},
    9: {'a_prendre': ['261', '271', '272', '25'], 'a_exclure': []},
    10: {'a_prendre': ['26', '27'], 'a_exclure': ['261', '271', '272', '25']},  
    11: {'a_prendre': ['16'], 'a_exclure': ['1688', '166']}, 
}

In [641]:
def get_aggregat_code(ligne):
    for code_aggregat, conditions in DEFINITION_AGGREGATS.items():
        if flag_aggregat(ligne, conditions['a_prendre'], conditions['a_exclure']):
            return code_aggregat

def flag_aggregat(ligne, a_prendre, a_exclure):
    flag = False
    for compte in a_prendre:
        if ligne.startswith(compte):
            flag = True
    for compte in a_exclure:
        if ligne.startswith(compte):
            flag=False
    return flag

def label_aggregat(code):
    if code:
        return AGGREGATS[code]

In [642]:
# créer les aggregats
data['code_aggregat'] = data['COMPTE'].apply(get_aggregat_code)
data = data.dropna(subset=['code_aggregat'])
data['aggregat'] = data['code_aggregat'].apply(label_aggregat)

In [643]:
# calculer les dépenses selon les aggrégats

In [644]:
calcul_depense_1 = data['code_aggregat'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
calcul_depense_2 = data['code_aggregat'].isin([7.0, 8.0, 9.0])
calcul_depense_3 = data['code_aggregat'].isin([10.0, 11.0])

data.loc[calcul_depense_1, 'depense'] = data['SD'] - data['SC']
data.loc[calcul_depense_2, 'depense'] = data['OBNETDEB'] - data['OBNETCRE'] + data['OOBCRE'] - data['OOBDEB'] 
data.loc[calcul_depense_3, 'depense'] = data['OBNETDEB']

In [ ]:
# étiquetter les fonctions

In [645]:
FONCTIONS = {
    0: 'Services généraux et opérations non ventilables',
    1: 'Sécurité et salubrité publiques',
    2: 'Enseignement – formation',
    3: 'Culture',
    4: 'Sport et jeunesse',
    5: 'Interventions sociales et santé',
    6: 'Famille',
    7: 'Logement',
    8: 'Aménagement et services urbains, environnement',
    9: 'Action économique',
}

data['FONCTION'].fillna(0, inplace=True)
data['code_fonction'] = data['FONCTION'].apply(lambda x: str(x)[0])
data['fonction'] = data['code_fonction'].apply(lambda x: FONCTIONS[int(x)])

In [646]:
# concat departement et code insee 
def pad_ndept(code):
    if len(str(code)) == 1:
        code = '0' + str(code)
    return str(code)

data['departement'] = data['NDEPT'].apply(pad_ndept)
data['INSEE'].fillna(0, inplace=True)
data['COM'] = data['departement'] + data['INSEE'].apply(lambda x: str(int(x)))

In [647]:
# pivoting 
index_cols = ['siren', 'IDENT', 'fonction', 'COM']
pivot_table = (
    pd
    .pivot_table(data_so, values='depense', index=index_cols, columns='aggregat', aggfunc='sum', fill_value=0)
    .reset_index()
)

In [651]:
# import and join pop data
pop_data = pd.read_csv('base_pop_insee.csv', sep=';')
pop_data_agg = pop_data.groupby(['COM', 'LIBCOM']).sum().sort_values('P16_POP', ascending=False).reset_index()
pivot_pop = pivot_table.merge(pop_data_agg, how='left', on='COM')

In [652]:
pivot_pop.to_csv('pivot_pop.csv')

In [653]:
pivot_pop_sample = pivot_pop[pivot_pop['COM'].apply(str).isin(['31555', '06088', '44109', '34172'])]

In [654]:
pivot_pop_sample.to_csv('pivot_pop_sample.csv')